In [1]:
%pip install mediapipe
%pip install tensorflow
# Add other necessary packages
%%python -m pip install mediapipe



[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.8/208.8 MB 11.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 21.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 17.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

UsageError: Line magic function `%%python` not found.


In [4]:
# import mediapipe as mp
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras import layers, models

import cv2
import mediapipe as mp
import os
import pandas as pd
import numpy as np

from mediapipe.tasks import python
from mediapipe.tasks.python import audio

# Initialize MediaPipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1)

# Function to perform facial landmarking on an image
def perform_facial_landmarking(image_path):
    # Read image
    image = cv2.imread(image_path)

    # Convert BGR to RGB
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process image
    results = face_mesh.process(rgb_image)

    # Extract landmarks
    landmarks = []
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            for landmark in face_landmarks.landmark:
                x, y, z = landmark.x, landmark.y, landmark.z
                landmarks.append([x, y, z])

    return np.array(landmarks)

# Folder containing FER2013 dataset images
dataset_folder = 'dataset/test'  # Adjust the folder path accordingly

# Apply facial landmarking to all images in subfolders
facial_landmarks_data = []

for emotion_folder in os.listdir(dataset_folder):
    emotion_path = os.path.join(dataset_folder, emotion_folder)
    print('Processing emotion:', emotion_folder)
    if os.path.isdir(emotion_path):
        for image_file in os.listdir(emotion_path):
            image_path = os.path.join(emotion_path, image_file)
            landmarks = perform_facial_landmarking(image_path)
            facial_landmarks_data.append({'landmarks': landmarks.tolist(), 'emotion': emotion_folder, 'image_path': image_path})

# Convert the result into a DataFrame
landmarks_df = pd.DataFrame(facial_landmarks_data)

# Save the DataFrame with facial landmarks, emotions, and image paths to a CSV file
landmarks_df.to_csv('fer2013_landmarks.csv', index=False)

I0000 00:00:1705160626.794650       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 83.1), renderer: Apple M2


Processing emotion: happy


KeyboardInterrupt: 

# Training the Model

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('landmarked_dataset.csv')

In [ ]:
# Pre-process the data
X = df.drop('emotion_label', axis=1).values
y = df['emotion_label'].values

In [ ]:
# Encode Labels
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
# Split the data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [ ]:
# Define the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


In [ ]:
# Evaluate the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


In [ ]:
# Make predictions
predictions = model.predict(X_test)
